<a href="https://colab.research.google.com/github/BenzterBit/DL-QSTP-Codes/blob/master/DCGAN(MODIFIED).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import nn,optim
from torch.autograd import Variable
from torchvision import transforms,datasets

In [0]:
pip install tensorboardx


In [0]:
from google.colab import files
files.upload()

In [0]:
from utils import Logger

In [0]:
transform =transforms.Compose( [
    transforms.Resize(64),transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))
])
trainset = datasets.MNIST(root='./data',train = True, download = True,transform = transform)
data_loader = torch.utils.data.DataLoader(trainset,batch_size = 128, shuffle = True )
num_batches = len(data_loader)

In [0]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator,self).__init__()
    in_features =1*28*28 
    out = 1
    
    self.hidden0 = nn.Sequential(
    nn.Conv2d(1,128,kernel_size=4, stride = 2 , padding = 1,bias=False),
    nn.LeakyReLU(0.2, inplace=True),
    nn.BatchNorm2d(128)
    )
    
    self.hidden1= nn.Sequential(
    nn.Conv2d(128,256,4,2,1,bias=False),
    nn.LeakyReLU(0.2, inplace=True),
    nn.BatchNorm2d(256)
    )
    
    self.hidden2 = nn.Sequential(
    nn.Conv2d(256,512,4,2,1,bias=False),
    nn.LeakyReLU(0.2, inplace=True),
    nn.BatchNorm2d(512)
    )
    
    self.hidden3 = nn.Sequential(
    nn.Conv2d(512,1024,4,2,1,bias=False),
    nn.LeakyReLU(0.2, inplace=True),
    nn.BatchNorm2d(1024)
    )
    
    self.out = nn.Sequential(
    nn.Conv2d(1024,1,4,1,0,bias=False),
    nn.Sigmoid()
    )
    
  def forward(self,x):
    out = self.hidden0(x)
    out = self.hidden1(out)
    out = self.hidden2(out)
    out = self.hidden3(out)
    out = self.out(out)
    return out

discriminator = Discriminator()
discriminator.cuda()

Discriminator(
  (hidden0): Sequential(
    (0): Conv2d(1, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (hidden1): Sequential(
    (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (hidden2): Sequential(
    (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (hidden3): Sequential(
    (0): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affi

In [0]:
class Generator(nn.Module):
  def __init__(self):
    super(Generator,self).__init__()
    
    self.hidden0 = nn.Sequential(
    nn.ConvTranspose2d(100,1024,kernel_size=4, stride = 1 , padding = 0,bias=False),
    nn.LeakyReLU(0.2, inplace=True),
    nn.BatchNorm2d(1024),
    nn.Dropout(0.3)
    )
    
    self.hidden1= nn.Sequential(
    nn.ConvTranspose2d(1024,512,4,2,1,bias=False),
    nn.LeakyReLU(0.2, inplace=True),
    nn.BatchNorm2d(512),
    nn.Dropout(0.3)
    )
    
    self.hidden2 = nn.Sequential(
    nn.ConvTranspose2d(512,256,4,2,1,bias=False),
    nn.LeakyReLU(0.2, inplace=True),
    nn.BatchNorm2d(256),
    nn.Dropout(0.3)
    )
    
    self.hidden3 = nn.Sequential(
    nn.ConvTranspose2d(256,128,4,2,1,bias=False),
    nn.LeakyReLU(0.2, inplace=True),
    nn.BatchNorm2d(128),
    nn.Dropout(0.3)
    )
    
    self.out = nn.Sequential(
    nn.ConvTranspose2d(128,1,4,2,1,bias=False),
    nn.Tanh()
    )
  def forward(self,x):
    out = self.hidden0(x)
    out = self.hidden1(out)
    out = self.hidden2(out)
    out = self.hidden3(out)
    out = self.out(out)
    return out

generator = Generator()
generator.cuda()
    
    

Generator(
  (hidden0): Sequential(
    (0): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3)
  )
  (hidden1): Sequential(
    (0): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3)
  )
  (hidden2): Sequential(
    (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3)
  )
  (hidden3): Sequential(
    (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): Lea

In [0]:
import numpy as np
def make_ones_sp(size):
  now = np.random.uniform(0.7,1.2)
  #torch_now = torch.from_numpy(now)
  ones = torch.zeros(size)
  ones+= now
  return Variable(ones.cuda())

def make_zeros_sp(size):
  now = np.random.uniform(0.0,0.3)
  #torch_now = torch.from_numpy(now)
  zeros = torch.zeros(size)
  zeros+= now
  return Variable(zeros.cuda())

def make_ones(size):
  data=Variable(torch.ones(size).cuda()) #do numpy random , and convert to pytorch .
  return data

def make_zeros(size):
  data = Variable(torch.zeros(size).cuda())
  return data

def noise(size):
  data = Variable(torch.randn(size,100).cuda())
  return data
def image_vector(image):
  vec = image.view(image.shape[0],-1)
  return vec
def vector_image(vector):
  image = vector.view(vector.shape[0],1,28,28)
  return image



In [0]:
d_optimiser = torch.optim.Adam(discriminator.parameters(),lr = 0.0002 , betas=(0.5, 0.999))
g_optimiser = torch.optim.Adam(generator.parameters(), lr =0.0002,betas=(0.5, 0.999))

In [0]:
loss = nn.BCELoss()


In [0]:
def calc2Norm():
  total_norm = 0
  total_norm2= 0
  for i in discriminator.parameters():
    
    param_norm = i.grad.data.norm(2)
    total_norm += param_norm.item() ** 2
  total_norm1 = total_norm ** (1. / 2)
  
  for p in generator.parameters():
    
    param_norm2 = p.grad.data.norm(2)
    total_norm2 += param_norm.item() ** 2
  total_norm3 = total_norm2 ** (1. / 2)
  
  return total_norm1,total_norm3
 

  
  

In [0]:
def train_discriminator(optimiser, real_data, fake_data):
  N = real_data.size(0)
  optimiser.zero_grad()
  #real data
  r_pred = discriminator(real_data).squeeze()
  
  r_err = loss(r_pred,make_ones_sp(N))
  
  r_err.backward()
  #fake data
  f_pred = discriminator(fake_data).squeeze() #D(G(z))
  
  f_err = loss(f_pred, make_zeros_sp(N))
  
  f_err.backward()
  
  optimiser.step()
  
  return r_err+f_err , r_pred, f_pred

def train_generator(optimiser , fake_data):
  N=fake_data.size(0)
  optimiser.zero_grad()
  
  f_pred = discriminator(fake_data).squeeze()
  f_err= loss(f_pred,make_ones_sp(N))
  f_err.backward()
  optimiser.step()
  return f_err

  
  

In [0]:
num_test_samples = 16
test_noise = noise(num_test_samples)

In [0]:
logger = Logger(model_name = 'DCGAN' , data_name = 'MNIST')
num_epoch = 30

for epoch in range(num_epoch):
  for n_batch, (real_batch,_) in enumerate(data_loader):
    N = real_batch.size(0)
    
    #train discriminator
    real_data = Variable(real_batch.cuda())
    fake_data = generator(noise(N).view(-1,100,1,1)).detach()
    
    d_err,d_r_pred,d_f_pred = train_discriminator(d_optimiser,real_data,fake_data)
    
    #train generator
    
    fake_data = generator(noise(N).view(-1,100,1,1)) #G(z)
    g_err = train_generator(g_optimiser,fake_data)
    
    # Log batch error
    logger.log(d_err, g_err, epoch, n_batch, num_batches)
    # Display Progress every few batches
    if (n_batch) % 100 == 0: 
        test_images = generator(test_noise.view(-1,100,1,1))
        test_images = test_images.data
        logger.log_images(
            test_images.cpu(), num_test_samples, 
            epoch, n_batch, num_batches
        );
        # Display status Logs
        logger.display_status(
            epoch, num_epoch, n_batch, num_batches,
            d_err, g_err, d_r_pred, d_f_pred
        )
        
        d_norm,g_norm = calc2Norm()
        print("D-Norm ",d_norm , " G-Norm ",g_norm)
    


NameError: ignored